In [1]:
import numpy as np
import pandas as pd
import joblib
import sys, os
sys.path.insert(0, '/home/monte.flora/python_packages/WoF_post')
sys.path.insert(0, '/home/monte.flora/python_packages/ml_workflow')
sys.path.insert(0, '/home/monte.flora/python_packages/wofs_ml_severe')

from wofs_ml_severe.io.load_ml_models import load_ml_model
from wofs.post.utils import load_yaml
import skexplain 
from skexplain.common.importance_utils import to_skexplain_importance 

lookup_file: /home/monte.flora/python_packages/WoF_post/wofs/data/psadilookup.dat


In [2]:
time = 'first_hour'
target = 'wind_severe_0km'
retro=False

ml_config = load_yaml(
    '/home/monte.flora/python_packages/wofs_ml_severe/wofs_ml_severe/conf/ml_config_realtime.yml')

parameters = {
                'target' : target,
                'time' : time, 
                'drop_opt' : '',
                'model_name' : 'Average',
                'ml_config' : ml_config,
            }

model_dict = load_ml_model(retro, **parameters)
model = model_dict['model']


ml_config = load_yaml(
    '/home/monte.flora/python_packages/wofs_ml_severe/wofs_ml_severe/conf/ml_config_retro.yml')

parameters['model_name'] = 'LogisticRegression'
parameters['ml_config'] = ml_config
model_dict = load_ml_model(retro, **parameters)
model = model_dict['model']
X =  model_dict['X']
y = model_dict['y']


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
explainer = skexplain.ExplainToolkit(('ML', model), X=X, y=y,)

In [ ]:
results = explainer.permutation_importance(
                                           n_vars=1, 
                                           evaluation_fn='norm_aupdc',
                                           n_permute=5, 
                                           subsample=0.1,
                                           n_jobs=1,
                                           verbose=True,
                                           random_seed=42, 
                                           direction='backward',
                                              )

Perm. Imp.:   0%|                                                                                                                                      | 0/1 [00:00<?, ?it/s]

In [ ]:
fig = explainer.plot_importance(data=results, 
                                panels=[('backward_singlepass', 'ML')], 
                                num_vars_to_plot=15, 
                                 )